# Iris Dataset Model Training and Logging with MLflow

This notebook demonstrates how to read the Iris dataset from a CSV file, train a logistic regression model, and log the training process and metrics to MLflow.

## Step 1: Load Data from CSV

First, we will load the Iris dataset from a CSV file.


In [ ]:
import pandas as pd

# Load data from CSV
df = pd.read_csv("iris_dataset.csv")
X = df.drop("target", axis=1)
y = df["target"]

# Split the data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


## Step 2: Train the Model

Next, we will train a logistic regression model using the training data.


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Define the model hyperparameters
params = {
    "solver": "lbfgs",
    "max_iter": 1000,
    "multi_class": "auto",
    "random_state": 8888,
}

# Train the model
lr = LogisticRegression(**params)
lr.fit(X_train, y_train)

# Predict on the test set
y_pred = lr.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")


## Step 3: Log to MLflow

Finally, we will log the model and its metrics to MLflow.


In [ ]:
import mlflow
from mlflow.models import infer_signature

# Set our tracking server uri for logging
mlflow.set_tracking_uri("http://10.100.198.240:5000")

# Create a new MLflow Experiment
mlflow.set_experiment("iris data")

# Start an MLflow run
with mlflow.start_run():
    # Log the hyperparameters
    mlflow.log_params(params)

    # Log the accuracy metric
    mlflow.log_metric("accuracy", accuracy)

    # Set a tag that we can use to remind ourselves what this run was for
    mlflow.set_tag("Training Info", "LR model trained from CSV")

    # Infer the model signature
    signature = infer_signature(X_train, lr.predict(X_train))

    # Log the model
    mlflow.sklearn.log_model(
        sk_model=lr,
        artifact_path="iris_model",
        signature=signature,
        input_example=X_train,
        registered_model_name="iris-data",
    )

    print("Logged run ID:", mlflow.active_run().info.run_id)
